In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date
import time
import json
import re
import tracker_functions as tf

# Test Function Steps - Collect Data

In [2]:
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument('--start-maximized')

browser = webdriver.Chrome(service=service, options=options)
browser.implicitly_wait(10)
browser.get('https://www.drogariaspacheco.com.br/mascara-facial-sallve-super-renovadora-50g/p')


###########################################################################################################

time.sleep(10)
product_page = BeautifulSoup(browser.page_source, 'html.parser')

name = product_page.find('div', attrs={'class':'productName'}).text

for number in range(50,70):

    try:
        ean_script = browser.find_element(By.XPATH, f'/html/head/script[{number}]').get_attribute('innerHTML')
        ean = re.search('"productEans":\["(?:[^"]|"")*"\]', ean_script).group(0)[16:-2]
    except:
        pass

notify_panel = product_page.find('div', attrs={'class':'notifyme-title-div','style':'display: none;'})

if notify_panel:
    price = product_page.find('strong', attrs={'class':'skuBestPrice'}).text[3:]
else:
    price = 'Esgotado'

###########################################################################################################

browser.quit()

print(name)
print(price)
print(ean)


# Test Function Steps - Webscraping

In [ ]:
browser = tf.initialize_browser('DSP',  headless=False)

#######################################################################################################################################


time.sleep(5)

WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))).click()

try:
    while browser.find_element(By.CLASS_NAME, 'progress-bar-prateleira'):
        load_products = browser.find_element(By.CLASS_NAME, 'btn-primary')
        browser.execute_script('arguments[0].click()', load_products)
        time.sleep(5)
except:
    pass

products = browser.find_elements(By.CLASS_NAME, 'collection-link')
    
menu_info = None
last_page = 0


#######################################################################################################################################


names_list = []
prices_list = []
eans_list = []
    
index = 0

for i in range(0, 100):
        
    try:
        
        print(f'Attempt: {i+1}')

        for product in range(index, len(products)):

            index = product

            if 'https://www.drogariasaopaulo.com.br/sallve' not in browser.current_url or 'https://www.drogariaspacheco.com.br/sallve' not in browser.current_url:
                time.sleep(2)
                browser.get('https://www.drogariasaopaulo.com.br/sallve')
                browser.refresh()

                try:
                    while browser.find_element(By.CLASS_NAME, 'progress-bar-prateleira'):
                        load_products = browser.find_element(By.CLASS_NAME, 'btn-primary')
                        browser.execute_script('arguments[0].click()', load_products)
                        time.sleep(5)
                except:
                    pass

            products = browser.find_elements(By.CLASS_NAME, 'collection-link')

            products[product].click()

            time.sleep(10)
            product_page = BeautifulSoup(browser.page_source, 'html.parser')

            name = product_page.find('div', attrs={'class':'productName'}).text

            for number_script in range(50,70):
            
                try:
                    ean_script = browser.find_element(By.XPATH, f'/html/head/script[{number_script}]').get_attribute('innerHTML')
                    ean = re.search('"productEans":\["(?:[^"]|"")*"\]', ean_script).group(0)[16:-2]
                except:
                    pass
                
            notify_panel = product_page.find('div', attrs={'class':'notifyme-title-div','style':'display: none;'})

            if notify_panel:
                price = product_page.find('strong', attrs={'class':'skuBestPrice'}).text[3:]
            else:
                price = 'Esgotado'

            names_list.append(name)
            prices_list.append(price)
            eans_list.append(ean)

            str_error = None  

    except Exception as e:
        str_error = str(e)

    if str_error:
        time.sleep(2)
            
    else:
        break

browser.quit()

In [ ]:
products_dict = {}

products_dict['drugstore'] = ['DVN' for x in range(0, len(names_list))]
products_dict['products_name'] = names_list
products_dict['price'] = prices_list
products_dict['EAN'] = eans_list

df_ws = pd.DataFrame(data=products_dict)
df_ws

In [1]:
import tracker_functions as tf

tf.track_price(['ECM'], headless=False)

Starting program at 2022-12-23 09:30:44.613349
Starting drugstore webscraping at 2022-12-23 09:30:44.613349
Initialized browser for ECM...
Menu information gathered successfully for ECM, starting webscraping...
Attempt: 1 - ECM
Attempt: 2 - ECM
Attempt: 3 - ECM
Attempt: 4 - ECM
Attempt: 5 - ECM
Attempt: 6 - ECM
Attempt: 7 - ECM
Attempt: 8 - ECM
Attempt: 9 - ECM
Attempt: 1 - ECM
Attempt: 2 - ECM
Attempt: 3 - ECM
Attempt: 4 - ECM
Attempt: 5 - ECM
Attempt: 6 - ECM
Attempt: 7 - ECM
Attempt: 8 - ECM
Attempt: 9 - ECM
Attempt: 10 - ECM
Attempt: 11 - ECM
Attempt: 12 - ECM
Attempt: 13 - ECM
Attempt: 14 - ECM
Attempt: 15 - ECM
Data have been collected successfully for ECM, storing data...
Data have been stored successfully for ECM
Execution time for ECM: 9.91 minutes
The data has been exported successfully in a csv file!
Total execution time: 9.91 minutes
